# Het enige verschil met #1 is de if statement, rij 23 van cell 6
_Nog elleganter is om bijna geheel automatisch te selecteren, hier geen ik een voorbeeld van een leden-zonder-date tool_

Om alle leden in een ketzer te mailen hebben we een aantal packages nodig. <br>
De meeste zitten standaard bij python, sommige niet: als je pandas of numpy niet hebt installeer die dan. <br>
Dat kan met: _pip install pandas_ en _pip install numpy_

In [1]:
# packages nodig voor mailen
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
import time

# data management == heel handig
import pandas as pd
import numpy as np

We beginnen met het ophalen van ons excel, stop je excel in de zelfde map en voer de naam in waar staat _Leden van activiteit.xlsx_ . <br>
Zo lang het ketzer systeem het zelfde blijft hebben we altijd de kollomen 'Unnamed: 0', 'Uid', 'Lidnaam', 'Opmerking', 'Opmerking2', 'Lid sinds','Aangemeld door'. Deze behouden we maar gebruiken voor _'lidnaam'_ voor aanhef en _'Uid'_ voor mail.<br>
we slaan de eerste rij over, deze is altijd 'C.S.R. Delft - Beheer activiteiten', dat is de skiprows[0]

In [2]:
df_in = pd.read_excel("Leden van activiteit.xlsx",skiprows=[0])
df_in.columns

Index(['Unnamed: 0', 'Uid', 'Lidnaam', 'Opmerking', 'Opmerking2', 'Lid sinds',
       'Aangemeld door'],
      dtype='object')

Ter controlle de excel zie je dan hieronder:

In [3]:
df_in

,Unnamed: 0,Uid,Lidnaam,Opmerking,Opmerking2,Lid sinds,Aangemeld door
0,NaN,x101,Jan Lid,NaN,NaN,2022-05-20 12:12,Jan Lid
1,NaN,x102,Jannette Lid,NaN,NaN,2022-05-20 12:12,Jannette Lid


Vervolgens laten we de lege kolom weg (unnamed:0) en kunnen we leden mailen met Uid@csrdelft.nl (ja vet handig)

In [4]:
# hij spuit een error maar dat is geen probleem in principe dus dit fixt het :)
import warnings
warnings.filterwarnings("ignore")

# behouden de kollomen
df = df_in[['Uid','Lidnaam','Opmerking','Opmerking2']]
# voegen een mail kollom aan
df['Mail'] = df['Uid'].apply(lambda x: f'{x}@csrdelft.nl')

df

,Uid,Lidnaam,Opmerking,Opmerking2,Mail
0,x101,Jan Lid,NaN,NaN,x101@csrdelft.nl
1,x102,Jannette Lid,NaN,NaN,x102@csrdelft.nl


Stel je wilt een deel los mailen, dan kan je makkelijkst selecteren op lid nummer.<br>
hier geef ik als voorbeeld df2, maar je kan hem gwn df noemen<br>
Bij echte uids kan je gwn een nummer ipv string invoeren<br>
Haal de twee # weg om te zien hoe het werkt:

In [5]:
if "Uid" in df.columns: # check om error te voorkomen
    df = df.set_index("Uid")
# df = df.loc[['x101']]
# df

Dan het echte verzenden:

In [6]:
############# Dingen om aan te passen #############
# deze is heel belangrijk, zorg dat hij bij testen op FALSE staat, anders verstuur je perongelijk mailtjes
send = False
# pas je commissie mail aan
sender_email = "commissie@csrdelft.nl"

############# Instellingen #############
# wat server instellingen belangrijk om verbinding te maken met de server, inprincipe generiek 
port = 587
smtp_server = "smtp.knorrie.org"

# prompt om je wachtwoord in te voergen: 
if send:    
    password = input("Type your password and press enter:\n")

# voor alle rijen in de excel
for i in range(len(df)):
    # per rij nemen we die specifieke
    row = df.iloc[i]
    
    ############# Deel wat anders is: #############
    # als de rij opmerkingen leeg is krijg je dus geen mail. 
    if pd.isna(row['Opmerking']):
        
        ############# Instellingen #############
        # maak een message opject aan 
        message = MIMEMultipart("alternative")
        
        # de verstuurder is de commissie
        message["From"] = sender_email

        # de ontavnger halen we uit de rij 
        receiver_email = row['Mail']
        message["To"] = receiver_email

        ############# Dingen om aan te passen #############
        #### gebruik bijv dit om de html opmaak te bouwen: https://html-online.com/editor/  of met de hand ####

        # geef de mail een title
        message["Subject"] = f"Voorbeeldmail naar alle leden in de activiteiten ketzer"
            # add main body
        body = f"""\
            <html>
              <body>
                    <p>Beste {row.Lidnaam}, <br></p>
                    <p>Het gala van 21 oktober komt er al snel aan!</p>
                    <p> Om alles af te kunnen ronden horen wij graag voor 12 augustus 00:12 2022 wie u date is.<br>
                    Dit klinkt misschien vroeg maar om de nodige voorbereidingen te kunnen treffen
                    hebben wij tijdig een overzicht nodig van de aanwezigen.<br>
                    Vooral ook omdat we graag de novieten verwelkomen op het gala. <br>
                    Schrijf dus vooral uw blauwe/roze brieven en zet u date er bij in de ketzer!
                    </p>              


                    <p>Met vriendelijke groet,
                    <br> De 12e LustrumCie der C.S.R. Delft</p>

              </body>
            </html>

            """
        ############# Verzenden #############
        # je voegt dan de html toe
        message.attach(MIMEText(body, "html"))

        # je kan ook een foto toevoegen bijv. 
        picture = False
        if picture:
            with open('Postergala.jpeg','rb') as fin:
                img = MIMEImage(fin.read())
                img.add_header('Content-Disposition', 'inline; filename=poster.jpg')
                message.attach(img)


        # Als je aan het testen ben kan je printen aan zetten en dan zie je een voorbeeld
        printing = True

        # dit verstuurd het daadwerkelijk
        if send:
            context = ssl.create_default_context()
            with smtplib.SMTP(smtp_server, port) as server:
                server.ehlo()  # Can be omitted
                server.starttls(context=context)
                server.ehlo()  # Can be omitted
                server.login(sender_email, password)
                server.sendmail(sender_email, receiver_email, message.as_string())

            print(f"{i}: {receiver_email} sent")

        # als het een test is print hij het het bericht
        else:
            if printing:
                print(message)

        # als je deze er niet in doet stopt ie na 100 mails
        time.sleep(1)

Content-Type: multipart/alternative; boundary="===============5865620434320512191=="
MIME-Version: 1.0
From: commissie@csrdelft.nl
To: x101@csrdelft.nl
Subject: Voorbeeldmail naar alle leden in de activiteiten ketzer

--===============5865620434320512191==
Content-Type: text/html; charset="us-ascii"
MIME-Version: 1.0
Content-Transfer-Encoding: 7bit

            <html>
              <body>
                    <p>Beste Jan Lid, <br></p>
                    <p>Het gala van 21 oktober komt er al snel aan!</p>
                    <p> Om alles af te kunnen ronden horen wij graag voor 12 augustus 00:12 2022 wie u date is.<br>
                    Dit klinkt misschien vroeg maar om de nodige voorbereidingen te kunnen treffen
                    hebben wij tijdig een overzicht nodig van de aanwezigen.<br>
                    Vooral ook omdat we graag de novieten verwelkomen op het gala. <br>
                    Schrijf dus vooral uw blauwe/roze brieven en zet u date er bij in de ketzer!
        

Hierboven een voorbeeld mail in tekst en dit heironder hoe het in gmail er uit ziet

![Hierboven een voorbeeld mail ](Voorbeeld.png)